# Exercise 2: Data Cleaning and Integration

We again require several libraries.

In [ ]:
# Import the required libraries
import tempfile
import sqlite3
import os
import urllib.request
import scipy.stats
import pandas as pd

Once again, we rely on the AdventureWorks database, but this time we load different data into our `DataFrames`.

In [ ]:
# Create a temporary directory
dataset_folder = tempfile.mkdtemp()

# Build path to database
database_path = os.path.join(dataset_folder, "adventure-works.db")

# Get the database
urllib.request.urlretrieve(
    "https://github.com/FAU-CS6/KDD-Databases/raw/main/AdventureWorks/adventure-works.db",
    database_path,
)

# Open connection to the adventure-works.db
connection = sqlite3.connect(database_path)

In [ ]:
# Modify the database to contain dirty data
cursor = connection.cursor()
cursor.executescript(
    "UPDATE Person SET LastName = 'Doe';"
    "UPDATE Employee SET HireDate = STRFTIME('%Y-%m-%d %H:%M:%S', HireDate);"
    "UPDATE Employee SET BirthDate = STRFTIME('%Y-%m-%d %H:%M:%S', BirthDate);"
    "UPDATE Employee SET Gender = 'Male' WHERE Gender = 'M' AND NationalIDNumber LIKE \"%8\";"
    "UPDATE Employee SET Gender = 'Female' WHERE Gender = 'F' AND NationalIDNumber LIKE \"%7%\";"
    "UPDATE Employee SET BirthDate = STRFTIME('%Y-%m-%d', BirthDate) WHERE NationalIDNumber LIKE \"%2%\";"
    "UPDATE Employee SET SickLeaveHours = 2306 WHERE BusinessEntityID = 10;"
    "UPDATE Employee SET VacationHours = -12 WHERE BusinessEntityID = 21;"
    "UPDATE Person SET LastName = 'Doe';"
    "UPDATE Employee SET JobTitle = 'None' WHERE NationalIDNumber LIKE \"%1\";"
    "UPDATE Employee SET CurrentFlag = 0 WHERE NationalIDNumber = 658797903 OR NationalIDNumber = 974026903;"
    "UPDATE Employee SET HireDate = STRFTIME('%Y-%m-%d %H:%M:%S', DATE(BirthDate, '-10 year')) "
    'WHERE NationalIDNumber LIKE "2%";'
)

In [ ]:
# Create the dirty DataFrame(s)
# Employee DataFrame
dirty_employee_df = pd.read_sql_query(
    "SELECT NationalIDNumber, LoginID, OrganizationNode, JobTitle, BirthDate, MaritalStatus, Gender,"
    "HireDate, SalariedFlag, VacationHours, SickLeaveHours, CurrentFlag, PersonType, NameStyle, Title,"
    "FirstName, MiddleName, LastName, Suffix, EmailPromotion, AdditionalContactInfo, Demographics "
    "FROM Employee "
    "JOIN Person ON Employee.BusinessEntityID = Person.BusinessEntityID",
    connection,
    index_col="NationalIDNumber",
)

Recall that we did not look for dirty data in exercise 1. 

Real world data rarely contains data without some inconsistencies or noise. For this reason, we will now look at the `dirty_employee_df` that contains obvious inconsistencies.

<div class="alert alert-block alert-info">

**Task 1:** 

Use the skills you learned to familiarize yourself with the `dirty_employee_df`. In doing so, try to identify as many problems as possible with the dataset at hand.

</div>

In [ ]:
# Use the methods you learned in Part One to familiarize yourself with dirty_employee_df
# (Create new code cells whenever necessary)

Incomplete data can take many different forms. If you look at the present data set, you will notice `None` values in various attributes. 

In [ ]:
# Print the columns containing at least one `None`
dirty_employee_df[
    [
        "OrganizationNode",
        "JobTitle",
        "Title",
        "MiddleName",
        "Suffix",
        "AdditionalContactInfo",
    ]
].head(25)

Not every `None` equates to missing data. Sometimes it is simply the correct information that the attribute value is "nothing". This can be seen in the six attributes presented. 

<div class="alert alert-block alert-info">

**Task 2:**

For each of the attributes at hand, consider whether the `None` values indicate incomplete information, or if `None` is justified.
</div>

Noisy data, i.e. small measurement inaccuracies, are difficult to detect in the context of such an exercise. However, it is almost certain that the present data set does not contain any noisy data. 

<div class="alert alert-block alert-info">

**Task 3:**
    
Consider why noisy data is unlikely to be included in `dirty_employee_df`.
</div>


Examples of inconsistencies of this data set can be found in the attributes `Gender`, `BirthDate`, and `HireDate`. 

<div class="alert alert-block alert-info">

**Task 4:**
    
Print the head of the attributes `Gender`, `BirthDate`, and `HireDate`.
</div>

In [ ]:
# Print the head of "Gender", "BirthDate", and "HireDate"

<div class="alert alert-block alert-info">

**Task 5:**
    
Consider what inconsistencies are in the `Gender` attribute.
</div>

<div class="alert alert-block alert-info">

**Task 6:**
    
Consider what inconsistencies are in the `BirthDate` attribute.</div>

<div class="alert alert-block alert-info">

**Task 7:**
    
Consider what inconsistencies are in the `HireDate` attribute. (Hint: Consider the attribute in conjunction with the other two attributes)</div>

Errors in numerical values and in the special outlier are sometimes not quite detectable at a glance. Fortunately, we have already learned methods in Part One that we can now use.

First, let's look at the `SickLeaveHours` attribute. Does it contain outliers or errors?

<div class="alert alert-block alert-info">

**Task 8:**
    
Use a boxplot diagram to graphically analyze whether `SickLeaveHours` contains Outliers.
</div>

In [ ]:
# Draw a boxplot diagram for "SickLeaveHours"

<div class="alert alert-block alert-info">

**Task 9:**
    
Think about a way to find out which tuple contains the outlier in `SickLeaveHours`.
</div>

In [ ]:
# Output the tuple containing the outlier

Additionally, let's take a look at `VacationHours`.

<div class="alert alert-block alert-info">

**Task 10:**
    
Use a boxplot diagram to graphically analyze whether `VacationHours` contains outliers.
</div>

In [ ]:
# Draw a boxplot diagram for "VacationHours"

<div class="alert alert-block alert-info">

**Task 11:**
    
Even though the boxplot diagram does not show any outliers, it clearly indicates an error in `VacationHours`. Which error?
</div>

<div class="alert alert-block alert-info">

**Task 12:**
    
Output the affected tuples.
</div>

In [ ]:
# Output the tuple(s) containing the error

There is also an intentional change to the original data in the `dirty_employee_df`. It can be found in either the `MaritalStatus`, `SalariedFlag`, `FirstName`, or `LastName` attribute.

<div class="alert alert-block alert-info">

**Task 13:** 
    
Again, independently use your learned skills to search the attributes `MaritalStatus`, `SalariedFlag`, `FirstName`, and `LastName` for the intentional change.
</div>

In [ ]:
# Search for the intentional change

<div class="alert alert-block alert-info">

**Task 14:**
    
In which of the attributes is the intentional error to be found.
</div>

The attribute with the intentional error: ?


<div class="alert alert-block alert-info">

**Task 15:**
    
Why do you think is this error intentional?
</div>

The mere detection of dirty data is, of course, only a first step in the data science process. While it is a best case scenario to correct dirty data step by step once it has been identified, this is often a lengthy and difficult process. 

In our example only inconsistencies in `Gender` and `BirthDate` can be quickly fixed.

<div class="alert alert-block alert-info">

**Task 16:**
    
Replace all occurrences of `Female` with `F` and all occurrences of `Male` with `M` in the `Gender` attribute of the `dirty_employee_df`. (Help: [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html))
</div>

In [ ]:
# Replace "Female" and "Male" values in "Gender"

<div class="alert alert-block alert-info">

**Task 17:**
    
Delete the suffix `00:00:00` in the `BirthDate` attribute. (Help: [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) - Hint: You may want to use a regular expression.)
</div>

In [ ]:
# Delete the suffix 00:00:00 in BirthDate

If only individual tuples contain an error/outlier and these cannot be manually fixed, the most efficient approach is often to simply remove these tuples from the dataset. 

For example this would apply to the tuples with `NationalIDNumber` 243322160 and 879342154 in the `dirty_employee_df`.

<div class="alert alert-block alert-info">

**Task 18:**
    
Delete the tuples with `NationalIDNumber` 243322160 and 879342154. (Help: [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html))
</div>

In [ ]:
# Delete the tuples with NationalIDNumber 243322160 and 879342154

Even for attributes that do not contain any information, it usually makes more sense to remove them from the data set. 

In this example this is the case with the attribute `LastName`.

<div class="alert alert-block alert-info">

**Task 19:**
    
Delete the attribute `LastName`. (Help: [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html))
</div>

In [ ]:
# Delete the attribute LastName

In the context of data integration, we mainly looked at correlation in the lecture. Calculation of this depends on the type of data.

One of the two data types we looked at in more detail in the lecture is nominal data. This describes all data that that is used to label variables without providing any quantitative value.

The first combination of nominal attributes we will look at in this section are `Gender` and `CurrentFlag`. We start by displaying the contingency table for these attributes.

<div class="alert alert-block alert-info">

**Task 20:**
    
Use the pandas function `crosstab()` to create a contingency table for the attributes `Gender` and `CurrentFlag`. Show the subtotals one time and once not. (Help: [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html))
</div>

In [ ]:
# Display a contingency table without subtotals

In [ ]:
# Display a contingency table with subtotals

The disadvantage of this contingency table is, of course, that only the observed quantities are displayed. For the calculation of the correlation, however, the expected quantities are also important. These can be calculated for example with the function `expected_freq()` from SciPy.

<div class="alert alert-block alert-info">

**Task 21:**
    
Use `expected_freq()` to output the expected quantities for the attributes `Gender` and `CurrentFlag`. (Help: [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.contingency.expected_freq.html#scipy.stats.contingency.expected_freq))
</div>

In [ ]:
# Display the expected quantities

One can see that both the expected and observed values are quite close indicating a small correlation. 

However, in the lecture, the Chi-squared test was presented as a method to validate this more accurately.

<div class="alert alert-block alert-info">

**Task 22:**
    
Use the function `chi2_contingency` from SciPy to determine the correlation between `Gender` and `CurrentFlag`. (Help: [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html#scipy.stats.chi2_contingency))
</div>

In [ ]:
# Compute chi-squared for "Gender" and "CurrentFlag"

<div class="alert alert-block alert-info">

**Task 23:**
    
Find out what the different values in the above output of `chi2_contingency` stand for and describe how to interpret them in this case.
</div>

Second, let's look at the `Gender` and `SalariedFlag` attributes.

<div class="alert alert-block alert-info">

**Task 24:**
    
Using the methods learned above, calculate Chi-squared for `Gender` and `SalariedFlag`.
</div>

In [ ]:
# Compute chi-squared for "Gender" and "SalariedFlag"

<div class="alert alert-block alert-info">

**Task 25:**
    
Interpret the chi-squared for `Gender` and `SalariedFlag`.
</div>

The other data type we have considered in the context of correlation is the numeric data type. Here, it is suitable to look at the connection between `VacationHours` and `SickLeaveHours`. 

A method of graphical analysis of correlation in numerical data should already be known from Exercise 1: 

<div class="alert alert-block alert-info">

**Task 26:**
    
Draw a scatter plot regarding `VacationHours` and `SickLeaveHours`.
</div>

In [ ]:
# Draw a scatter plot regarding "VacationHours" and "SickLeaveHours"

However, what was not part of the method in Exercise 1 is the interpretation of this diagram.

<div class="alert alert-block alert-info">

**Task 27:**
    
Interpret the scatter plot regarding `VacationHours` and `SickLeaveHours`.
</div>

We can, of course, evaluate this graphical analysis mathematically. In the lecture we used Pearson's product-moment coefficient for this purpose.

<div class="alert alert-block alert-info">

**Task 28:**
    
Compute pearson's product-moment coefficient for `VacationHours` and `SickLeaveHours`. Use SciPy's `pearsonr()` function. (Help: [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html))
</div>

In [ ]:
# Compute pearson's product-moment coefficient for "VacationHours" and "SickLeaveHours"

<div class="alert alert-block alert-info">

**Task 29:**
    
Find out what the different values in the above output of `pearsonr` stand for and describe how to interpret them in this case.
</div>